In [12]:
import cv2
import os
from pathlib import Path

In [13]:
# generate dataset

def generateDataset(img, userid,img_id):

    # creating a folder for a user
    Path("data/user_{}".format(userid)).mkdir(parents=True,exist_ok=True)

    # convert to gray and resize before saving
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray=cv2.resize(gray,(100,100))

    # writing images in the user folder
    savingPath="data/user_"+str(userid)+"/user_"+str(userid)+"."+str(img_id)+".jpg"
    cv2.imwrite(savingPath,gray)
    print("Images recorded successfully!")

In [14]:
# draw boundary
def drawBoundary(img,classifier,scaleFactor,minNeighbour,color,txt):
#     convert to gray
    grayImg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     detect feature in image
    features=classifier.detectMultiScale(grayImg,scaleFactor,minNeighbour)
    coordinates=[]
    for (x,y,w,h) in features:
#         draw rectangle
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)

        #label rectangle
        cv2.putText(img,txt,(x+3,y-3),cv2.FONT_HERSHEY_PLAIN,0.9,color,1,cv2.LINE_AA)
        
        coordinates=[x,y,w,h]
    return coordinates,img



# method to detect face
def detectFace(img,faceClassifier,img_id):
    # Rimg,match=recocgnize(img,clf,faceCascade,1.1,6,(185,200,120))
    
    # if (not match):
    coordinates,img= drawBoundary(img,faceClassifier,1.1,6,(105,250,10),'My Face')
    print(len(coordinates))
    if (len(coordinates)==4):
        roi_img=img[coordinates[1]:coordinates[1]+coordinates[3],coordinates[0]:coordinates[0]+coordinates[2]]
        print(roi_img)
        user_id=1
        generateDataset(roi_img,user_id,img_id)
        
            
    return img
    

In [15]:
# main code area
faceCascade= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

video_capture=cv2.VideoCapture(0);
video_capture.set(10,150)

img_id=0
while True:
    _,img=video_capture.read()
    img=detectFace(img,faceCascade,img_id)
    cv2.imshow("Face detection",img)
    
    img_id+=1
    if cv2.waitKey(1) & 0xFF == ord('z'):
        break
# release webcam
video_capture.release()
cv2.destroyAllWindows()

4
[[[105 250  10]
  [105 250  10]
  [105 250  10]
  ...
  [105 250  10]
  [105 250  10]
  [105 250  10]]

 [[105 250  10]
  [105 250  10]
  [105 250  10]
  ...
  [105 250  10]
  [105 250  10]
  [105 250  10]]

 [[105 250  10]
  [105 250  10]
  [  3  24  34]
  ...
  [ 21  19   0]
  [ 22  22   0]
  [105 250  10]]

 ...

 [[105 250  10]
  [105 250  10]
  [ 14   6   4]
  ...
  [ 36  23   3]
  [ 35  22   3]
  [105 250  10]]

 [[105 250  10]
  [105 250  10]
  [  9   3   6]
  ...
  [ 37  22   3]
  [ 37  22   3]
  [105 250  10]]

 [[105 250  10]
  [105 250  10]
  [105 250  10]
  ...
  [105 250  10]
  [105 250  10]
  [105 250  10]]]
Images recorded successfully!
4
[[[105 250  10]
  [105 250  10]
  [105 250  10]
  ...
  [105 250  10]
  [105 250  10]
  [105 250  10]]

 [[105 250  10]
  [105 250  10]
  [105 250  10]
  ...
  [105 250  10]
  [105 250  10]
  [105 250  10]]

 [[105 250  10]
  [105 250  10]
  [  0   0   0]
  ...
  [ 24  13   0]
  [ 27  12   0]
  [105 250  10]]

 ...

 [[105 250  10]
  